In [ ]:
import numpy as np
import os
import gc
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import roc_auc_score
import random
import tensorflow as tf

In [ ]:
epochs = 1
size = 100
len_batches = 500
users = os.listdir('users_folder')
while len(users)%len_batches != 0:
    users.append(random.choice(users))
users = np.array(users)
users_in_batch = int(len(users)/len_batches)

In [ ]:
def get_batch(array):
    batch = pd.read_csv('users_folder/{}'.format(array[0]))
    for i in range(1, len(array)):
        batch = pd.concat([batch, pd.read_csv('users_folder/{}'.format(array[i]))])
    batch_y = batch['20'].fillna(0)
    batch_x = np.expand_dims(batch.drop('20', axis = 1).values, 1)
    return batch_x, batch_y

In [ ]:
def fit(model, x, y):
    model.train_on_batch(x, y)
    return model

In [ ]:
def score(model, x, y):
    local_result = roc_auc_score(y, model.predict(x))
    local_weights = model.get_weights()
    return local_result, local_weights

In [ ]:
def get_model():
    model = tf.keras.Sequential([ 
        tf.keras.layers.LSTM(size, input_shape = ( 1, 19)),
        tf.keras.layers.Dense(200, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')

    ])
    model.compile(loss='binary_crossentropy', optimizer='Adam',metrics=['Accuracy'])
    return model

In [ ]:
best_result = 0
val_size = 0.25
best_weights = None
model = get_model()
val = random.choices(users, k = int(len(users)*val_size))
x_val, y_val = get_batch(val)
for _ in range(epochs):
    print('эпоха номер {}'.format(_))
    np.random.shuffle(users)
    users_in_batches = users.reshape((len_batches, users_in_batch))
    for i in tqdm(users_in_batches):
        batch_x, batch_y = get_batch(i)
        model = fit(model, batch_x, batch_y)
    local_result, local_weights = score(model, x_val, y_val)
    if best_result < local_result:
        best_weights = local_weights
        best_result = local_result
    else:
        model.set_weights(best_weights)
    print(best_result)